# Lab session 9: Imbalanced dataset
__Name: Makesh Srinivasan<br>
Registration number: 19BCE1717<br>
Course code: CSE4020<br>
Faculty: Dr. Abdul Quadir<br>
Slot: L31 + L32<br>
Date: 12-October-2021 Monday__<br><br>
    
Instructions:<br>
Imbalanced dataset - perform classification on the car_evaluation dataset, then perform the same on over-sampled and under-sampled dataset

# Imbalanced dataset

__Libraries and packages__

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

__Load the dataset__

The dataset car_evaluation is imported from the UCI repository as shown below. There are six features and one label (outcome). The column names are added to provide description of the attribute as shown below

In [2]:
def load(URL_):
    data = pd.read_csv(URL_, header = None)
    data.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'outcome']
    print("Data loaded...")
    return data
data = load('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data')
data.tail()

Data loaded...


buying maint  doors persons lug_boot safety outcome
1723    low   low  5more    more      med    med    good
1724    low   low  5more    more      med   high   vgood
1725    low   low  5more    more      big    low   unacc
1726    low   low  5more    more      big    med    good
1727    low   low  5more    more      big   high   vgood

__Values of the label - Outcome__

In [3]:
data.outcome.value_counts()

unacc    1210
acc       384
good       69
vgood      65
Name: outcome, dtype: int64

__Splitting__

In [4]:
X = data.drop(['outcome'],axis=1)
y = data['outcome']
X.head()

buying  maint doors persons lug_boot safety
0  vhigh  vhigh     2       2    small    low
1  vhigh  vhigh     2       2    small    med
2  vhigh  vhigh     2       2    small   high
3  vhigh  vhigh     2       2      med    low
4  vhigh  vhigh     2       2      med    med

__Encode the data__

Encode all the string datatype to integer values

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
enc = LabelEncoder()
X.loc[:, ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']] = X.loc[:, ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']].apply(enc.fit_transform)
X.head()

buying maint doors persons lug_boot safety
0      3     3     0       0        2      1
1      3     3     0       0        2      2
2      3     3     0       0        2      0
3      3     3     0       0        1      1
4      3     3     0       0        1      2

__KNN Classifier__

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)
model = KNeighborsClassifier()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)
print("Accuracy = ", accuracy_score(y_test, y_predict))
pd.crosstab(y_test, y_predict)

Accuracy =  0.9267822736030829


col_0    acc  good  unacc  vgood
outcome                         
acc       84     1     17      0
good       8    13      0      0
unacc      3     0    368      0
vgood      4     2      3     16

KNN classifier on an imbalanced dataset gave an accuracy of 0.926 and the following are the class-wise accuracy in prediction. <br>
1) acc: 84/102 -> 82% <br>
2) good: 13/21 -> 61% <br>
3) unacc: 368/371 -> 99% <br>
4) vgood: 16/25 -> 64% <br>

<hr>

# SOLUTIONS: Sampling on imbalanced dataset

There are three ways to work on imbalanced dataset and they are as follows.

1) <a href="#oversample">Over-sampling</a><br>
2) <a href="#undersample">Under-sampling</a><br>
3) <a href="#smote">SMOTE</a><br>

<hr>

## <div id="oversample">1) Oversampling</div>

__Libraries__

In [8]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

__Overfitting the dataset__

In [9]:
over=RandomOverSampler()
Over_X,Over_y=over.fit_resample(X,y)

In [10]:
print(Counter(Over_y))
Over_y

Counter({'unacc': 1210, 'acc': 1210, 'vgood': 1210, 'good': 1210})


0       unacc
1       unacc
2       unacc
3       unacc
4       unacc
        ...  
4835    vgood
4836    vgood
4837    vgood
4838    vgood
4839    vgood
Name: outcome, Length: 4840, dtype: object

__KNN Classifier__

In [11]:
train_x, test_x, train_y, test_y = train_test_split(Over_X,Over_y, test_size=0.3, random_state=10)
model = KNeighborsClassifier()
model.fit(train_x,train_y)
y_predict = model.predict(test_x)

print("Accuracy = ", accuracy_score(test_y,y_predict))
pd.crosstab(test_y,y_predict)

Accuracy =  0.8836088154269972


col_0    acc  good  unacc  vgood
outcome                         
acc      310    32      0     21
good       0   348      0      0
unacc     76    21    259     19
vgood      0     0      0    366

The imbalanced dataset was augmented using oversampling and the KNN classifier produced an accuracy of 0.88 <br>
As shown above in the table, the true positives are higher. <br>
1) acc: 310/363 -> 85%<br>
2) good: 348/348 -> 100%<br>
3) unacc: 259/375 -> 69%<br>
4) vgood: 366/366 -> 100%<br>

<hr>

## <div id="undersample">2) Undersampling</div>

__Libraries__

In [12]:
from imblearn.under_sampling import RandomUnderSampler

__Underfitting the dataset__

In [13]:
under=RandomUnderSampler()
Under_X,Under_y=under.fit_resample(X,y)
print(Counter(Under_y))

Counter({'acc': 65, 'good': 65, 'unacc': 65, 'vgood': 65})


In [14]:
Under_y

0        acc
1        acc
2        acc
3        acc
4        acc
       ...  
255    vgood
256    vgood
257    vgood
258    vgood
259    vgood
Name: outcome, Length: 260, dtype: object

__KNN Classifier__

In [15]:
train_x, test_x, train_y, test_y = train_test_split(Under_X,Under_y, test_size=0.3, random_state=10)
model = KNeighborsClassifier()
model.fit(train_x,train_y)
y_predict = model.predict(test_x)

print("Accuracy = ", accuracy_score(test_y,y_predict))
pd.crosstab(test_y,y_predict)

Accuracy =  0.7051282051282052


col_0    acc  good  unacc  vgood
outcome                         
acc        7     7      0      8
good       0    16      0      0
unacc      2     4     10      1
vgood      0     0      1     22

The imbalanced dataset was augmented using undersampling and the KNN classifier produced an accuracy of 0.70 <br>
1) acc: 7/22 -> 32%<br>
2) good: 16/16 -> 100%<br>
3) unacc: 10/17 -> 58%<br>
4) vgood: 22/23 -> 96%<br>

<hr>

## <div id="smote">3) SMOTE</div>

__Libraries__

In [16]:
from imblearn.over_sampling import SMOTE

__SMOTE__

In [17]:
smote=SMOTE()
SMOTE_X,SMOTE_y=smote.fit_resample(X,y)
print(Counter(SMOTE_y))

Counter({'unacc': 1210, 'acc': 1210, 'vgood': 1210, 'good': 1210})


In [18]:
SMOTE_y

0       unacc
1       unacc
2       unacc
3       unacc
4       unacc
        ...  
4835    vgood
4836    vgood
4837    vgood
4838    vgood
4839    vgood
Name: outcome, Length: 4840, dtype: object

__KNN Classifier__

In [19]:
train_x, test_x, train_y, test_y = train_test_split(SMOTE_X,SMOTE_y, test_size=0.3, random_state=10)
model = KNeighborsClassifier()
model.fit(train_x,train_y)
y_predict = model.predict(test_x)

print("Accuracy = ", accuracy_score(test_y,y_predict))
pd.crosstab(test_y,y_predict)

Accuracy =  0.981404958677686


col_0    acc  good  unacc  vgood
outcome                         
acc      353     4      2      4
good       0   348      0      0
unacc     17     0    358      0
vgood      0     0      0    366

### Using SMOTE the imbalanced dataset was augmented and the KNN classifier produced an accuracy of 0.98 <br>
As shown above in the table, the true positives are higher.<br>
1) acc: 353/363 -> 96%<br>
2) good: 348/348 -> 100%<br>
3) unacc: 358/375 -> 96%<br>
4) vgood: 366/366 -> 100%<br>

<hr>